In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import html
from lxml import etree

In [153]:
url = 'https://www.rotowire.com/baseball/daily_lineups.htm'
page = requests.get(url)

In [154]:
tree = html.fromstring(page.content)

# Scraping
___
## Get boxes

In [185]:
class daily_lineups(object):
    def __init__(self):
        self.lineup_url = 'https://www.rotowire.com/baseball/daily_lineups.htm'
        self.page = requests.get(self.lineup_url)
        self.tree = html.fromstring(self.page.content)
        self.info_locations = {'team':"//div[@class='lineup is-mlb'][{}]/div[2]/div/div/div[{}]/div",
                               'pitchers':"//div[@class='lineup is-mlb'][{}]/div[2]/div[2]/ul[{}]/li[0]/div",
                               'players':"//div[@class='lineup is-mlb'][{}]/div[2]/div[2]/ul[{}]/li/a"}
    def main(self):
        games = []
        games_today = len(self.tree.xpath("//div[@class='lineup is-mlb']"))
        teams = [1,2]
        for game_num in range(games_today):
            try:
                game = {}
                for team in teams:
                    team_temp = tree.xpath(self.info_locations['team'].format('game','team'))
                    game['team{}'.format(team)] = self.get_team(team_temp)
                    pitcher_temp = tree.xpath(self.info_locations['pitcher'].format('game','team'))
                    game['pitcher{}'.format(team)] = self.get_pitcher(pitcher_temp)
                    player_temp = tree.xpath(self.info_locations['players'].format('game','team'))
                    game['players{}'.format(team)] = self.get_player(player_temp)
                games.append(game)
            except:
                
        return games
        

    def get_team(self, team_scrape):
        if type(team_scrape) == list:
            for t in team_scrape:
                print(t.text) 
        team = team_scrape.text
        return team
    def get_pitcher(self, pitcher_scrape):
        pitcher = pitcher_scrape.text
        return pitcher
    def get_players(self, players_scrape):
        player_list = []
        for player in players_scrape:
            player_list.append(player.attrib['title'])
        return player_list

In [186]:
gl = daily_lineups()

In [187]:
gl.main()

AttributeError: 'list' object has no attribute 'text'

In [189]:
gl.tree.xpath(gl.info_locations['team'].format('game','team'))

[]